## Logging クックブック

https://docs.python.org/ja/3/howto/logging-cookbook.html


In [6]:
import copy
import logging


def filter(record: logging.LogRecord):
    record = copy.copy(record)
    record.user = "jim"
    return record


logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter("%(message)s from %(user)-8s")
handler.setFormatter(formatter)
handler.addFilter(filter)
logger.addHandler(handler)

logger.info("A log message")


--- Logging error ---
Traceback (most recent call last):
  File "/Users/takeshiiijima/opt/anaconda3/lib/python3.9/logging/__init__.py", line 434, in format
    return self._format(record)
  File "/Users/takeshiiijima/opt/anaconda3/lib/python3.9/logging/__init__.py", line 430, in _format
    return self._fmt % record.__dict__
KeyError: 'user'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/takeshiiijima/opt/anaconda3/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/Users/takeshiiijima/opt/anaconda3/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/Users/takeshiiijima/opt/anaconda3/lib/python3.9/logging/__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "/Users/takeshiiijima/opt/anaconda3/lib/python3.9/logging/__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "/Users/t

In [2]:
import logging


# コンテキストを持たせるためのクラス
class ContextLogger(logging.LoggerAdapter):
    def process(self, msg, kwargs):
        # メッセージにコンテキスト情報を追加
        return f"{self.extra} - {msg}", kwargs


# ロガーのセットアップ
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
base_logger = logging.getLogger("my_logger")

# コンテキスト付きロガーを作成
context = {"user": "name", "trace_id": "12345"}
logger = ContextLogger(base_logger, context)

# ログ出力
logger.info("This is a log message.")
logger.info("Another message.")


2024-11-17 08:44:33,319 - my_logger - INFO - {'user': 'name', 'trace_id': '12345'} - This is a log message.
2024-11-17 08:44:33,321 - my_logger - INFO - {'user': 'name', 'trace_id': '12345'} - Another message.


In [9]:
import logging


class DynamicContextFilter(logging.Filter):
    def __init__(self):
        super().__init__()
        self.context = {}

    def add_context(self, **new_context):
        # コンテキストを更新
        self.context.update(new_context)

    def filter(self, record):
        # コンテキスト情報を LogRecord に追加
        for key, value in self.context.items():
            setattr(record, key, value)
        return True


# ロガーのセットアップ
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - [user: %(user)s] - [session_id: %(session_id)s] - %(message)s",
)
logger = logging.getLogger("my_logger")

# フィルターを作成してロガーに追加
context_filter = DynamicContextFilter()
logger.addFilter(context_filter)

# 初期ログ出力
context_filter.add_context(user="Takeshi")
logger.info("Initial message.")

# コンテキストを追加してログ出力
context_filter.add_context(session_id="12345")
logger.info("Session started.")

# さらにコンテキストを追加
context_filter.add_context(transaction_id="67890")
logger.info("Transaction processed.")


2024-11-17 08:43:55,208 - my_logger - INFO - Initial message.
2024-11-17 08:43:55,210 - my_logger - INFO - Session started.
2024-11-17 08:43:55,211 - my_logger - INFO - Transaction processed.
